<a href="https://colab.research.google.com/github/SueliHora/challenge2-Desafio_Telecom_X_Parte_2/blob/main/analise_churn_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Telecom X - Parte 2: Previsão de Churn com Machine Learning

## 1. Introdução

O propósito deste projeto é desenvolver e testar modelos de **Machine Learning** para antecipar a evasão de clientes (churn) em uma empresa de telecomunicações. Com base nos dados preparados na Parte 1, o foco está em organizar os dados para modelagem, treinar diferentes algoritmos, comparar seus desempenhos e, por fim, identificar os fatores mais relevantes que levam um cliente a cancelar o serviço.


In [ ]:
# --- 1. Importação e Carregamento dos Dados ---
import pandas as pd
df = pd.read_csv('/dados_tratados.csv')

# --- 2. Pré-processamento ---
df = df.drop('customerID', axis=1)
df_encoded = pd.get_dummies(df, drop_first=True)

# --- 3. Separação de Features (X) e Alvo (y) ---
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# --- 4. Divisão em Dados de Treino e Teste ---
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 5. Scaling dos Dados (NOVO PASSO!) ---
from sklearn.preprocessing import StandardScaler

# Cria o objeto scaler
scaler = StandardScaler()

# "Aprende" a escala com os dados de TREINO e os transforma
X_train_scaled = scaler.fit_transform(X_train)

# Apenas transforma os dados de TESTE com a mesma escala aprendida
X_test_scaled = scaler.transform(X_test)


print("Dados preparados, divididos e escalados com sucesso!")

Dados preparados, divididos e escalados com sucesso!


## 2. Treinamento e Avaliação dos Modelos

Nesta fase, dois algoritmos de classificação foram desenvolvidos e testados com o objetivo de identificar qual apresenta o melhor desempenho na previsão de clientes que irão cancelar o serviço (churn).


In [ ]:
# --- Modelo 1: Regressão Logística ---
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Instancia o modelo (max_iter não é mais tão crítico, mas podemos manter)
modelo_logistico = LogisticRegression(max_iter=1000)

# ATENÇÃO: Usando os dados escalados (scaled) para treinar
modelo_logistico.fit(X_train_scaled, y_train)

# E usando os dados de teste escalados para prever
previsoes_logistico = modelo_logistico.predict(X_test_scaled)

print("--- Resultados do Modelo de Regressão Logística ---")
print(f"Acurácia: {accuracy_score(y_test, previsoes_logistico) * 100:.2f}%")
print("\nRelatório de Classificação:")
print(classification_report(y_test, previsoes_logistico))

--- Resultados do Modelo de Regressão Logística ---
Acurácia: 79.60%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1033
           1       0.64      0.54      0.58       374

    accuracy                           0.80      1407
   macro avg       0.74      0.71      0.72      1407
weighted avg       0.79      0.80      0.79      1407



In [ ]:
# --- Modelo 2: Random Forest ---
from sklearn.ensemble import RandomForestClassifier

# Instancia e treina o modelo
modelo_floresta = RandomForestClassifier(random_state=42)
modelo_floresta.fit(X_train, y_train)

# Faz as previsões e avalia o modelo
previsoes_floresta = modelo_floresta.predict(X_test)

print("--- Resultados do Modelo Random Forest ---")
print(f"Acurácia: {accuracy_score(y_test, previsoes_floresta) * 100:.2f}%")
print("\nRelatório de Classificação:")
print(classification_report(y_test, previsoes_floresta))

--- Resultados do Modelo Random Forest ---
Acurácia: 77.75%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1033
           1       0.61      0.46      0.52       374

    accuracy                           0.78      1407
   macro avg       0.71      0.68      0.69      1407
weighted avg       0.76      0.78      0.77      1407



## 3. Análise do Modelo Selecionado e Principais Fatores

Após a avaliação comparativa, a **Regressão Logística** foi eleita como o modelo principal, por apresentar melhor desempenho no **recall** da classe “churn”.  
A seguir, analisaremos quais variáveis (features) tiveram maior influência nas decisões do modelo.


In [ ]:
# Extrai os coeficientes do modelo de Regressão Logística
feature_names = X.columns
coeficientes = modelo_logistico.coef_[0]

# Cria um DataFrame para visualizar a importância de cada fator
importancia_features = pd.DataFrame({'Fator': feature_names, 'Peso': coeficientes})

# Ordena os fatores pela sua importância (valor absoluto do peso)
importancia_features['Importancia'] = importancia_features['Peso'].abs()
importancia_ordenada = importancia_features.sort_values(by='Importancia', ascending=False)
importancia_ordenada = importancia_ordenada.drop(columns='Importancia')

print("--- Fatores que Mais Influenciam o Churn (do maior para o menor impacto) ---")
print("Pesos positivos aumentam a chance de Churn. Pesos negativos diminuem a chance de Churn.")
print(importancia_ordenada.head(15))

--- Fatores que Mais Influenciam o Churn (do maior para o menor impacto) ---
Pesos positivos aumentam a chance de Churn. Pesos negativos diminuem a chance de Churn.
                             Fator      Peso
1                           tenure -1.504375
3                    Charges.Total  0.762711
25               Contract_Two year -0.611861
10     InternetService_Fiber optic  0.549261
2                  Charges.Monthly -0.445468
24               Contract_One year -0.297896
28  PaymentMethod_Electronic check  0.181594
26            PaperlessBilling_Yes  0.176807
9                MultipleLines_Yes  0.176481
21                 StreamingTV_Yes  0.155072
23             StreamingMovies_Yes  0.152699
13              OnlineSecurity_Yes -0.146230
19                 TechSupport_Yes -0.120072
0                    SeniorCitizen  0.087943
6                   Dependents_Yes -0.079903


## 4. Conclusão e Recomendações Estratégicas

### Resumo do Projeto
O propósito deste projeto foi construir um modelo de **machine learning** capaz de prever a evasão de clientes (churn) na **Telecom X** e identificar os fatores mais relevantes que influenciam essa decisão.

### Desempenho e Seleção do Modelo
Foram avaliados dois algoritmos de classificação: **Regressão Logística** e **Random Forest**.  
O modelo escolhido como principal foi a **Regressão Logística**, por apresentar melhor desempenho na tarefa mais crítica para a empresa: detectar clientes com risco real de churn.  
O modelo atingiu um **recall de 54%** para os clientes propensos a cancelar, enquanto o Random Forest alcançou 46%.

### Principais Insights
A análise do modelo vencedor destacou os fatores que mais impactam a evasão e a retenção:  
- Clientes com **internet de Fibra Óptica** e que utilizam **meios de pagamento e fatura digital** apresentam maior probabilidade de cancelar.  
- Fatores que fortalecem a retenção incluem a adesão a **contratos de longo prazo (1 ou 2 anos)** e a contratação de serviços adicionais como **Suporte Técnico** e **Segurança Online**.

### Recomendações Estratégicas
Com base nos achados, sugerem-se as seguintes ações:  
1. **Fidelização de Contratos:** Desenvolver campanhas que incentivem clientes de planos mensais a migrar para contratos de 1 ou 2 anos, oferecendo descontos ou vantagens explícitas.  
2. **Análise do Serviço de Fibra:** Realizar pesquisas detalhadas com clientes de Fibra Óptica para entender os motivos da alta evasão — se relacionados a preço, qualidade do serviço ou concorrência intensa.  
3. **Pacotes com Valor Agregado:** Promover serviços de **Suporte Técnico** e **Segurança Online** como diferenciais para aumentar a fidelização, já que se mostraram eficazes na retenção de clientes.
